In [17]:
import os
from typing import Union, Optional, Tuple

import numpy as np
import pandas as pd

# ASE imports
from ase.build import bulk, stack
from ase.lattice.cubic import BodyCenteredCubic as bcc

# Pyiron workflow imports
import pyiron_workflow as pwf
from pyiron_workflow import Workflow
from pyiron_workflow_atomistics.dataclass_storage import CalcInputMinimize
from pyiron_workflow_atomistics.bulk import optimise_cubic_lattice_parameter
from pyiron_workflow_atomistics.bulk_defect.vacancy import get_vacancy_formation_energy
from pyiron_workflow_atomistics.calculator import calculate_structure_node
from pyiron_workflow_atomistics.surface.builder import create_surface
from pyiron_workflow_lammps.engine import LammpsEngine

# Pymatgen imports
from pymatgen.core import Structure
from pymatgen.io.ase import AseAtomsAdaptor

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Example for LAMMPS engine

In [58]:
inp = CalcInputMinimize()
inp.relax_cell = False
Engine = LammpsEngine(EngineInput = inp)

Engine.working_directory = "surface_energy"
Engine.lammps_log_filepath = "minimize.log"
# Adjust this line to match your LAMMPS installation
# If you are using conda-lammps or a LAMMPS binary on your system, you can use directly:
Engine.command = "lmp -in in.lmp -log minimize.log"
Engine.input_script_pair_style = "eam/fs"
Engine.path_to_model = os.getcwd() + "/Al-Fe.eam.fs"

from pyiron_workflow_atomistics.surface.surface_study import calculate_surface_energy

wf = Workflow("surface_energy", delete_existing_savefiles=True)
# /root/github_dev/pyiron_workflow_atomistics/pyiron_workflow_atomistics
structure = bulk("Fe", a=2.85, cubic=True)
wf.opt_cubic_cell = optimise_cubic_lattice_parameter(
    structure=structure,
    name="Fe",
    crystalstructure="bcc",
    calculation_engine=Engine,
    parent_working_directory="opt_cubic_cell",
    rattle=0.1,
    strain_range=(-0.02, 0.02),
    num_points=10,
    eos_type="birchmurnaghan",
)
# Any eos_type supported by ase.eos.EquationOfState can be used,
# e.g. sjeos, taylor, murnaghan, birch, birchmurnaghan, pouriertarantola, vinet, antonschmidt, p3
wf.surface_energy = calculate_surface_energy(symbol = "Fe",
                                            calculation_engine = Engine,
                                            miller_indices = (1, 1, 1),
                                            min_length = 50,
                                            vacuum = 10.0,
                                            crystalstructure = "bcc",
                                            mu_bulk = None,
                                            a = wf.opt_cubic_cell.outputs.a0,
                                            cubic = True)
wf.run()

2025-09-26 01:20:24,567 - pyiron_log - WARNING - Directory 'test_opt_cubic_cell/strain_000' already exists. Existing files may be overwritten.


current mode  minimize


2025-09-26 01:20:26,196 - pyiron_log - WARNING - Directory 'test_opt_cubic_cell/strain_001' already exists. Existing files may be overwritten.
2025-09-26 01:20:27,790 - pyiron_log - WARNING - Directory 'test_opt_cubic_cell/strain_002' already exists. Existing files may be overwritten.
2025-09-26 01:20:29,398 - pyiron_log - WARNING - Directory 'test_opt_cubic_cell/strain_003' already exists. Existing files may be overwritten.
2025-09-26 01:20:31,011 - pyiron_log - WARNING - Directory 'test_opt_cubic_cell/strain_004' already exists. Existing files may be overwritten.
2025-09-26 01:20:32,631 - pyiron_log - WARNING - Directory 'test_opt_cubic_cell/strain_005' already exists. Existing files may be overwritten.
2025-09-26 01:20:34,235 - pyiron_log - WARNING - Directory 'test_opt_cubic_cell/strain_006' already exists. Existing files may be overwritten.
2025-09-26 01:20:35,844 - pyiron_log - WARNING - Directory 'test_opt_cubic_cell/strain_007' already exists. Existing files may be overwritten.

{'opt_cubic_cell__equil_struct': Atoms(symbols='Fe2', pbc=True, cell=[2.8553600861257875, 2.8553600861257875, 2.8553600861257875], initial_magmoms=...),
 'opt_cubic_cell__B': np.float64(177.0390163507032),
 'opt_cubic_cell__equil_energy_per_atom': np.float64(-4.012966860824359),
 'opt_cubic_cell__equil_volume_per_atom': np.float64(11.639991349321107),
 'opt_cubic_cell__volumes': [np.float64(21.787771257000003),
  np.float64(22.08554991529631),
  np.float64(22.386029495370348),
  np.float64(22.689222191000006),
  np.float64(22.99514019596296),
  np.float64(23.303795704037036),
  np.float64(23.615200909),
  np.float64(23.92936800462963),
  np.float64(24.24630918470369),
  np.float64(24.56603664300001)],
 'opt_cubic_cell__structures': [Atoms(symbols='Fe2', pbc=True, cell=[[2.7929999999999997, 1.7102192550092785e-16, 1.7102192550092785e-16], [0.0, 2.7929999999999997, 1.7102192550092785e-16], [0.0, 0.0, 2.7929999999999997]]),
  Atoms(symbols='Fe2', pbc=True, cell=[[2.805666666666667, 1.7179

In [ ]:
print(f"EAM: {np.round(wf.opt_cubic_cell.outputs.a0.value, 4)} Å")
print(f"Surface energy: {wf.surface_energy.surface_energy.value/16.021766208:.3f} eV/Å² or {wf.surface_energy.surface_energy.value:.2f} J/m²")
print(f"n_atoms: {len(wf.surface_energy.outputs.relaxed_surface.value)}")
import warnings
from IPython.display import display, HTML

warnings.warn("The 111 surface energy is not correct for this potential, it is too low, dft predicts 2.73 J/m², see http://crystalium.materialsvirtuallab.org/")
display(HTML('<div style="color:orange; font-weight:bold;">⚠️ The 111 surface energy is not correct for this potential, it is too low, DFT predicts 2.73 J/m², see <a href="http://crystalium.materialsvirtuallab.org/" target="_blank">crystalium.materialsvirtuallab.org</a></div>'))

EAM: 2.8553
Surface energy: 0.062 eV/Å² or 0.99 J/m²
n_atoms: 38
